In [30]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score


- เราจะใช้ข้อมูลหลายขนาด เพื่อแสดงให้เห็นว่าการทำ Nested Cross-Validation นั้นสามารถปรับค่า hyperparameters และประเมินโมเดลได้ดีกว่าการทำ Cross-Validation ธรรมดา

In [37]:
# Load data based on user input size
n = int(input("Enter the size of your data: "))
if n == 20:
    print("Results of data size 20")
    data = pd.read_csv('/Users/gunnviryasiri/Documents/CDTI4/ml/myowncode/Data/HeightWeight20.csv')
elif n == 100:
    print("Results of data size 100")
    data = pd.read_csv('/Users/gunnviryasiri/Documents/CDTI4/ml/myowncode/Data/HeightWeight100.csv')
elif n == 10000:
    print("Results of data size 10000")
    data = pd.read_csv('/Users/gunnviryasiri/Documents/CDTI4/ml/myowncode/Data/HeightWeight.csv')
else:
    print("Enter the correct data size")
    exit()

X = data[['Height']].values
y = data['Weight'].values

Results of data size 20


In [38]:
# ฟังก์ชันสำหรับคำนวณความเที่ยงตรง (Precision) และความแม่นยำ (Accuracy)
def evaluate_model(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    return rmse, r2

In [40]:
# Cross Validation
def cross_validation(X, y, k=10):
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    
    rmse_values = []
    r2_values = []
    
    model = LinearRegression()  # กำหนดโมเดลให้ชัดเจน
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        model.fit(X_train, y_train)
        
        y_pred = model.predict(X_test)
        
        rmse, r2 = evaluate_model(y_test, y_pred)
        rmse_values.append(rmse)
        r2_values.append(r2)
    
    return np.mean(rmse_values), np.mean(r2_values)

### ใน Nested Cross-Validation จะมีการแบ่งข้อมูลออกเป็นสองลูป:

1. ลูปภายนอก (Outer loop): ใช้สำหรับการประเมินโมเดล
2. ลูปภายใน (Inner loop): ใช้สำหรับการปรับค่า hyperparameters ของโมเดล

In [41]:
# Nested Cross Validation
def nested_cross_validation(X, y, outer_k=5, inner_k=3):
    outer_kf = KFold(n_splits=outer_k, shuffle=True, random_state=42)
    
    outer_rmse_values = []
    outer_r2_values = []

    for train_index, test_index in outer_kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # กำหนดโทเดลและค่าให้ parameter
        model = LinearRegression()
        param_grid = {}  # LinearRegression ไม่มี hyperparameters ที่จะค้นหา
        
        # Inner loop 
        inner_kf = KFold(n_splits=inner_k, shuffle=True, random_state=42)
        grid_search = GridSearchCV(model, param_grid, cv=inner_kf, scoring='neg_mean_squared_error')
        grid_search.fit(X_train, y_train)
        
        # model ที่ดีสุด
        best_model = grid_search.best_estimator_
        
        # Evaluate ของ outer
        y_pred = best_model.predict(X_test)
        rmse, r2 = evaluate_model(y_test, y_pred)
        
        outer_rmse_values.append(rmse)
        outer_r2_values.append(r2)
    
    return np.mean(outer_rmse_values), np.mean(outer_r2_values)

- ค่า R2 ที่ใกล้เคียงกับ 1 มากที่สุดแสดงว่าโมเดลสามารถอธิบายความแปรปรวนของข้อมูลได้ดี ค่า R2 ที่ต่ำหรือเป็นลบแสดงว่าโมเดลไม่สามารถอธิบายความแปรปรวนได้ดี

- ค่า RMSE ที่ต่ำกว่าหมายถึงโมเดลที่มีความแม่นยำมากกว่า

In [42]:
# Run nested cross-validation
outer_k = 5
inner_k = 3

rmse_nested, r2_nested = nested_cross_validation(X, y, outer_k=outer_k, inner_k=inner_k)
print(f"Nested Cross-Validation - RMSE: {rmse_nested:.4f}, R2: {r2_nested:.4f}")

# Run cross-validation
rmse_cv, r2_cv = cross_validation(X, y, k=10)
print(f"Cross-Validation - RMSE: {rmse_cv:.4f}, R2: {r2_cv:.4f}")

Nested Cross-Validation - RMSE: 7.5564, R2: 0.7963
Cross-Validation - RMSE: 7.2139, R2: -1.1526


In [43]:
# Run nested cross-validation
outer_k = 5
inner_k = 3

rmse_nested, r2_nested = nested_cross_validation(X, y, outer_k=outer_k, inner_k=inner_k)
print(f"Nested Cross-Validation - RMSE: {rmse_nested:.4f}, R2: {r2_nested:.4f}")

# Run cross-validation
rmse_cv, r2_cv = cross_validation(X, y, k=10)
print(f"Cross-Validation - RMSE: {rmse_cv:.4f}, R2: {r2_cv:.4f}")

Nested Cross-Validation - RMSE: 7.5564, R2: 0.7963
Cross-Validation - RMSE: 7.2139, R2: -1.1526
